In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn import svm
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import cross_validate, KFold, train_test_split

In [2]:
data = pd.read_csv('train.csv')
# test_data = pd.read_csv('test.csv') # тестовый набор нельзя использовать: нет колонки SalePrice
# позже раздробим data на обучающуюся и тестовую выборки

In [3]:
yfull = data.SalePrice
Xfull = data.drop( labels = ["SalePrice"], axis = 1)


In [4]:
# data = data.dropna(axis='columns')
#Находим категориальные признаки
cat_feat = list(Xfull.dtypes[Xfull.dtypes == object].index)

#отфильтруем непрерывные признаки
num_feat = [f for f in Xfull if f not in (cat_feat + ['Id'])]

# Создаем дамми-переменные для категорий
X_dummies = pd.get_dummies(Xfull[cat_feat], columns=cat_feat)
# dummy_test = pd.get_dummies(X_test[cat_feat], columns=cat_feat)

dummy_cols = list(set(X_dummies))

X_dummies = X_dummies[dummy_cols]
# dummy_test = dummy_test[dummy_cols]

# Заменяем пропуски на специальное значение -999, чтобы деревья могли их отличить
Xfull = pd.concat([Xfull[num_feat].fillna(-999),
                     X_dummies], axis=1)


In [5]:
# Выделяем holdout-выборку 
X, Xhold, y, yhold = train_test_split(
    Xfull, yfull, random_state=42
)
X = X.reindex()
Xhold = Xhold.reindex()
y = y.reindex()
yhold = yhold.reindex()

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1095 entries, 1023 to 1126
Columns: 288 entries, MSSubClass to Exterior2nd_VinylSd
dtypes: float64(3), int64(33), uint8(252)
memory usage: 586.0 KB


In [7]:
# беру один фолд из 10 и строю на нем случайный лес, вывожу важность признаков.
# эдакая тренировка перед стекингом
# чуть поправил взятое отсюда
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold
kf = KFold(n_splits=10)
for train_index, test_index in kf.split(Xfull):
    rf = RandomForestRegressor(verbose=False, n_jobs=-1)
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = Xfull.iloc[train_index], Xfull.iloc[test_index]
    y_train, y_test = yfull[train_index], yfull[test_index]
    rf.fit(X_train,y_train)
    print("Random forest score:")
    print(rf.score(X_test,y_test))
    print("Feature importances")
    print(rf.feature_importances_)
    break # берем только один эксперимент, чтобы не заниматься никаким ансамблированием сейчас
# если нужны усредненные по k-фолдам значимости параметров, можно сделать усреднением. Но не вижу смысла.

Random forest score:
0.8650248449381439
Feature importances
[9.55092268e-04 5.01800561e-03 1.29389183e-02 5.83822050e-01
 4.73260540e-03 8.70348794e-03 5.60615553e-03 8.93385735e-03
 2.55760034e-02 5.74228122e-04 4.72527261e-03 3.80991674e-02
 2.24507359e-02 4.10264963e-02 1.24024770e-04 9.95921774e-02
 8.82134924e-04 2.12991959e-04 1.29417593e-02 5.26220994e-04
 2.27322258e-03 7.74499918e-04 4.63390927e-03 2.78586632e-03
 5.42355121e-03 2.36135935e-02 1.44762049e-02 4.17664974e-03
 4.13030472e-03 6.47650125e-04 2.23528486e-04 8.81171491e-04
 4.66632496e-04 4.19029024e-05 2.72921783e-03 1.38044315e-03
 5.72954917e-05 5.82879802e-04 1.92239268e-04 8.52331903e-06
 4.93259800e-06 7.55983442e-05 0.00000000e+00 1.67512103e-03
 3.62616122e-06 3.78500082e-06 1.50096538e-05 3.31927750e-06
 2.06734364e-05 1.96090260e-05 1.80269905e-05 1.27051216e-05
 9.41537539e-05 1.85416050e-05 6.16715837e-08 5.66048659e-05
 0.00000000e+00 2.42165568e-06 2.72811976e-04 3.60682174e-05
 2.03741161e-06 5.9125189

In [8]:
# кроссвалидация одного только случайного леса - переработаный код отсюда:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
cv_results = cross_validate(rf, Xfull, yfull, cv=10)
# sorted(cv_results.keys())
all_scores = cv_results['test_score']
from statistics import mean
# средний score по 10и фолдам для случайного леса:
mean(all_scores)
# тоже смысла не вижу - случайный лес внутри уже делит выборку 
# в итоге у каждого из деревьев совсем мало данных, 
# оттого, наверное, и score ниже чем у простого случайного леса ниже

0.8625083162376667

In [9]:
# Один случайный лес - учим на всем, кроме holdout, проверяем на holdout.
# Плюс выводим важность признаков
# Интересно, насколько скор будет отличаться у cv и такого подхода 
one_rf = RandomForestRegressor(verbose=False, n_jobs=-1)
one_rf.fit(X,y)
print(one_rf.score(Xhold,yhold))
print(one_rf.feature_importances_)
# скор получился тут су

0.8909908042149817
[1.70844584e-03 7.12353395e-03 1.66682379e-02 5.52650675e-01
 3.63226063e-03 1.08022404e-02 6.70129322e-03 3.31433498e-03
 2.75278496e-02 5.47870424e-04 5.25203538e-03 3.36189223e-02
 2.65815432e-02 2.64522503e-02 9.30487104e-05 1.18613762e-01
 8.66153121e-04 7.81756296e-05 2.67507898e-03 8.34392569e-04
 1.30884392e-03 3.81299543e-04 8.67458792e-03 4.93596504e-03
 7.32163190e-03 2.63805052e-02 1.46945850e-02 5.38280473e-03
 4.95373940e-03 8.23253341e-04 2.23397829e-04 2.41848775e-03
 2.03330567e-03 6.13724076e-05 3.95380827e-03 1.32477589e-03
 2.97689031e-04 8.32226872e-04 4.76812218e-04 1.87011894e-05
 4.14514534e-06 4.53554395e-05 2.47005356e-05 2.07866823e-03
 1.10903238e-05 1.86234801e-06 1.95216815e-05 7.59781989e-05
 1.57870110e-04 2.23997567e-05 1.83416397e-05 5.92043053e-06
 8.35628654e-05 4.07039895e-05 2.49078929e-07 2.33293997e-04
 0.00000000e+00 5.10091500e-07 2.10123319e-04 5.10677769e-05
 8.58689023e-09 3.46502112e-09 3.48280530e-04 3.77320416e-04
 3.98

In [17]:
# на трех моделях строим стекинг, 
# а потом считаем скор каждой из них в отдельности
# непонятно, как правильно совмещать кросвалидацию и holdout. 
# для меня это два независимых способа оценки качества обучения.estimators

# отсюда код взят и доработан
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html?highlight=regressor#sklearn.ensemble.StackingRegressor
from sklearn.linear_model import RidgeCV, SGDRegressor, LinearRegression
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

# X, y = load_diabetes(return_X_y=True)
estimators = [
    ('rcv', RidgeCV()),
    ('svr', LinearSVR(random_state=42)),
    ('rf', RandomForestRegressor(random_state=42)),
    # ('lr', LinearRegression())
    # ('sgd', SGDRegressor(random_state=42))
]
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RidgeCV(),
    cv=10,
)

reg.fit(X, y).score(Xhold, yhold)


C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinea

0.9013787553552302

In [54]:
# непонятно, как правильно совмещать кросвалидацию и holdout. 
# для меня это два независимых способа оценки качества обучения.estimators 

# кроссвалидация
for est in estimators:
    cv_results = cross_validate(est[1], Xfull, yfull, cv=10)
    print("Estimator: ")
    print(est[0])
    print("cross_val score: ")
    print(cv_results['test_score'])
    print("cross_val mean score: ")
    print(mean(cv_results['test_score']))
    # print()

Estimator: 
rcv
cross_val score: 
[0.91089043 0.90884634 0.89996691 0.78113865 0.88449541 0.88597101
 0.86924554 0.88814094 0.56787695 0.88265082]
cross_val mean score: 
0.84792230118371
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the n

In [16]:
# сравнение на holdout-выборке
for est in estimators:
    the_est = est[1]
    the_est.fit(X,y)
    print("Estimator: ")
    print(est[0])
    print(the_est.score(Xhold,yhold))
    # print(one_rf.feature_importances_)  
    # print("cross_val score: ")
    # print(cv_results['test_score'])
    # print("cross_val mean score: ")
    # print(mean(cv_results['test_score']))
    # # print()

Estimator: 
rcv
0.8807907617616811
C:\Users\Sergei\anaconda3\envs\uoit\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
Estimator: 
svr
0.803096227449928
Estimator: 
rf
0.8860695399769015


В первой попытке RidgeCV при сравнении на holdout дал бОльший score, чем стекинг.
Значит стекинг был построен неверно. Преподаватель посоветовал ставить сложные модели на первый слой (в estimators), а модель второго уровня брать попроще.
В результате стэкинг дал score 0.9, а все estimators отдельно - меньшие score.